In [11]:
from numpy import zeros, uint8

In [13]:
z = zeros((20,10,160,120), dtype=uint8)

In [ ]:
%load_ext Cython

In [23]:
%%cython
# distutils: language = c++

from numpy cimport ndarray, uint8_t
from numpy import copy, asarray, ascontiguousarray, uint8, float32, dstack, ndarray
from cython cimport boundscheck, wraparound
ctypedef unsigned char uchar
cdef class CopyNumpyArray:
    cdef:
        uchar[:,:,:,:,:] img
        int d,d1
    def __cinit__(self,uchar[:,:,:,:] img, int num ):
        self.d = img.shape[0]
        self.d1 = img.shape[1]
        self.copy_arrays(img,num )
    @boundscheck(False)  # Deactivate bounds checking
    @wraparound(False)
    cdef void copy_arrays(self, uchar[:,:,:,:] img, int num):
        
        cdef: 
            int i,j,k
            uchar[:,:] copied
        dim= []
        for i in range(self.d):
            dim1=[]
            for j in range(self.d1):
                dim2=[]
                for _ in range(num):
                    copied = img[i,j,:,:].copy()
                    dim2.append(copied)
                dim1.append(dim2)
            dim.append(dim1)
        self.img = asarray(dim, dtype=uint8)
    def get_array(self):
        if(self.img.shape[0]==0):
            return "the array is empty"
        else:
            return asarray(self.img, dtype=float)
                    
                    
        
        

In file included from /usr/local/lib/python3.8/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1969,
                 from /usr/local/lib/python3.8/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.8/dist-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /home/cristian/.cache/ipython/cython/_cython_magic_9beb82827a9b371c956e0c77d3d47efc.cpp:661:
/usr/local/lib/python3.8/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~


In [25]:
im = CopyNumpyArray(z, 7)
sample = im.get_array()
sample.dtype

dtype('float64')

In [2]:
%load_ext Cython

In [5]:
%%cython
# distutils: language = c++

from libcpp.vector cimport vector
from libc.string cimport memcpy
from libc.stdlib cimport malloc, free
from numpy cimport ndarray, uint8_t
from numpy import copy, asarray, ascontiguousarray, uint8, float32, dstack, ndarray
from opencv_mat cimport *
ctypedef unsigned char uchar


cdef class CopyNumpyArray:
    cdef:
        vector[vector[vector[Mat]]] mat_arrays
        int dim,dim1,r,c,n_copies
    def __cinit__(self, uchar[:,:,:,:] data, int copies):
        self.dim = data.shape[0]
        self.dim1 = data.shape[1]
        self.r = data.shape[2]
        self.c = data.shape[3]
        self.n_copies
        self.copy_imgArray(data, copies)

    cdef Mat np2Mat2D(self, uchar[:,:] data):
        assert (data.ndim==2), "ASSERT::1 channel grayscale only!!"

        cdef ndarray[uint8_t, ndim=2, mode ='c'] np_buff = ascontiguousarray(data, dtype=uint8)
        cdef unsigned int* im_buff = <unsigned int*> np_buff.data

        cdef Mat m
        m.create(self.r, self.c, CV_8UC1)
        memcpy(m.data, im_buff, self.r*self.c)
        return m
    cdef void copy_imgArray(self, uchar[:,:,:,:] data, int c):
        cdef:
            int i,j
            #Mat img
            Mat copied
            vector[vector[Mat]] imgs
            vector[Mat] copies

        for i in range(self.dim):
            for j in range(self.dim1):
                img = self.np2Mat2D(data[i,j,:,:])
                for _ in range(c):
                    #img.copyTo(copied)
                    copies.push_back(img)
                imgs.push_back(copies)
            self.mat_arrays.push_back(imgs)

    cdef object Mat2np(self, Mat m):
        # Create buffer to transfer data from m.data
        cdef Py_buffer buf_info
        # Define the size / len of data
        cdef size_t len = m.rows*m.cols*m.elemSize()#m.channels()*sizeof(CV_8UC3)
        # Fill buffer
        PyBuffer_FillInfo(&buf_info, NULL, m.data, len, 1, PyBUF_FULL_RO)
        # Get Pyobject from buffer data
        Pydata  = PyMemoryView_FromBuffer(&buf_info)

        # Create ndarray with data
        # the dimension of the output array is 2 if the image is grayscale
        if m.channels() >1 :
            shape_array = (m.rows, m.cols, m.channels())
        else:
            shape_array = (m.rows, m.cols)

        if m.depth() == CV_32F :
            ary = ndarray(shape=shape_array, buffer=Pydata, order='c', dtype=float32)
        else :
        #8-bit image
            ary = ndarray(shape=shape_array, buffer=Pydata, order='c', dtype=uint8)

        if m.channels() == 3:
            # BGR -> RGB
            ary = dstack((ary[...,2], ary[...,1], ary[...,0]))

        # Convert to numpy array
        pyarr = asarray(ary)
        return pyarr

    cdef ndarray[uint8_t, ndim=4] vec2np(self):
        cdef i,j,k
        arrays = []
        for i in range(self.dim):
            seq = []
            for j in range(self.dim1):
                copies=[]
                for k in range(self.n_copies):
                    copies.append(self.Mat2np(self.mat_arrays[i][j][k]))
                seq.append(copies)
            arrays.append(seq)
        return asarray(arrays, dtype=uint8)
    def get_arrays(self):
        return self.vec2np()

In file included from /usr/local/lib/python3.8/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1969,
                 from /usr/local/lib/python3.8/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.8/dist-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /home/cristian/.cache/ipython/cython/_cython_magic_6e2ac8578228145531fe158db7205135.cpp:657:
/usr/local/lib/python3.8/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~


ImportError: /home/cristian/.cache/ipython/cython/_cython_magic_6e2ac8578228145531fe158db7205135.cpython-38-x86_64-linux-gnu.so: undefined symbol: _ZN2cv8fastFreeEPv